In [13]:
import sys
!{sys.executable} -m pip install pandas numpy matplotlib seaborn scikit-learn


  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)
Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl (222 kB)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\SASA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [16]:
# =====================================================================
# BAGIAN 1: PERSIAPAN DATA DAN PREPROCESSING
# =====================================================================

class DataPreprocessor:
    """
    Kelas untuk melakukan preprocessing data depression dataset
    """
    
    def __init__(self, data_path):
        self.data_path = data_path
        self.data = None
        self.processed_data = None
    
    def load_data(self):
        """Memuat data dari file CSV"""
        self.data = pd.read_csv(self.data_path)
        print("Data berhasil dimuat!")
        print(f"Shape data: {self.data.shape}")
        return self.data
    
    def explore_data(self):
        """Eksplorasi data untuk memahami struktur dan distribusi"""
        print("\n=== EKSPLORASI DATA ===")
        print("\nInfo Dataset:")
        print(self.data.info())
        
        print("\nSample Data:")
        print(self.data.head())
        
        print("\nDistribusi Target (Depression):")
        print(self.data['Depression'].value_counts())
        
        # Visualisasi distribusi fitur utama
        self._plot_feature_distributions()
    
    def _plot_feature_distributions(self):
        """Membuat visualisasi distribusi fitur"""
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle('Distribusi Fitur Dataset Depression', fontsize=16)
        
        # Sleep Duration
        self.data['Sleep Duration'].value_counts().plot(kind='bar', ax=axes[0,0])
        axes[0,0].set_title('Sleep Duration')
        axes[0,0].tick_params(axis='x', rotation=45)
        
        # Dietary Habits
        self.data['Dietary Habits'].value_counts().plot(kind='bar', ax=axes[0,1])
        axes[0,1].set_title('Dietary Habits')
        
        # Academic Pressure
        axes[0,2].hist(self.data['Academic Pressure'].dropna(), bins=10, edgecolor='black')
        axes[0,2].set_title('Academic Pressure')
        
        # Study Hours
        axes[1,0].hist(self.data['Study Hours'].dropna(), bins=10, edgecolor='black')
        axes[1,0].set_title('Study Hours')
        
        # Financial Stress
        axes[1,1].hist(self.data['Financial Stress'].dropna(), bins=10, edgecolor='black')
        axes[1,1].set_title('Financial Stress')
        
        # Depression (Target)
        self.data['Depression'].value_counts().plot(kind='bar', ax=axes[1,2])
        axes[1,2].set_title('Depression (Target)')
        
        plt.tight_layout()
        plt.show()
    
    def preprocess_data(self):
        """
        Preprocessing data:
        1. Konversi variabel kategorik ke numerik
        2. Normalisasi data
        3. Pembentukan fitur gabungan
        """
        print("\n=== PREPROCESSING DATA ===")
        
        # Copy data untuk preprocessing
        processed = self.data.copy()
        
        # 1. Konversi Sleep Duration ke numerik
        sleep_mapping = {
            'Less than 5 hours': 1,
            '5-6 hours': 2,
            '7-8 hours': 3,
            'More than 8 hours': 4
        }
        processed['Sleep_Numeric'] = processed['Sleep Duration'].map(sleep_mapping)
        
        # 2. Konversi Dietary Habits ke numerik
        diet_mapping = {
            'Unhealthy': 1,
            'Moderate': 2,
            'Healthy': 3
        }
        processed['Diet_Numeric'] = processed['Dietary Habits'].map(diet_mapping)
        
        # 3. Konversi Suicidal Thoughts ke numerik
        processed['Suicidal_Numeric'] = processed['Have you ever had suicidal thoughts ?'].map({'No': 0, 'Yes': 1})
        
        # 4. Konversi Family History ke numerik
        processed['Family_History_Numeric'] = processed['Family History of Mental Illness'].map({'No': 0, 'Yes': 1})
        
        # 5. Konversi target Depression ke numerik
        processed['Depression_Numeric'] = processed['Depression'].map({'No': 0, 'Yes': 1})
        
        # 6. Pembentukan Fitur Gabungan sesuai konsep yang Anda tentukan
        
        # POLA HIDUP = f(Sleep Duration, Dietary Habits, Suicidal Thoughts)
        processed['Lifestyle_Score'] = (
            processed['Sleep_Numeric'] * 0.4 +
            processed['Diet_Numeric'] * 0.4 +
            (1 - processed['Suicidal_Numeric']) * 0.2  # Inversi karena suicidal thoughts negatif
        )
        
        # TEKANAN AKADEMIK = f(Academic Pressure, Study Hours)
        # Normalisasi Study Hours ke skala 1-5
        study_hours_normalized = (processed['Study Hours'] - processed['Study Hours'].min()) / (processed['Study Hours'].max() - processed['Study Hours'].min()) * 4 + 1
        processed['Academic_Stress_Score'] = (
            processed['Academic Pressure'] * 0.6 +
            study_hours_normalized * 0.4
        )
        
        # PREDISPOSISI DEPRESI = f(Financial Stress, Family History)
        processed['Depression_Predisposition'] = (
            processed['Financial Stress'] * 0.7 +
            processed['Family_History_Numeric'] * 1.5  # Family history memiliki bobot lebih tinggi
        )
        
        self.processed_data = processed
        print("Preprocessing selesai!")
        
        # Tampilkan statistik fitur gabungan
        print("\nStatistik Fitur Gabungan:")
        feature_cols = ['Lifestyle_Score', 'Academic_Stress_Score', 'Depression_Predisposition']
        print(processed[feature_cols].describe())
        
        return processed

In [17]:
# =====================================================================
# BAGIAN 2: DEFINISI FUNGSI MEMBERSHIP FUZZY
# =====================================================================

class FuzzyMembershipFunctions:
    """
    Kelas untuk mendefinisikan fungsi keanggotaan fuzzy
    """
    
    @staticmethod
    def trimf(x, a, b, c):
        """Fungsi keanggotaan triangular"""
        return np.maximum(0, np.minimum((x - a) / (b - a), (c - x) / (c - b)))
    
    @staticmethod
    def trapmf(x, a, b, c, d):
        """Fungsi keanggotaan trapezoidal"""
        return np.maximum(0, np.minimum(np.minimum((x - a) / (b - a), 1), (d - x) / (d - c)))
    
    @staticmethod
    def gaussmf(x, mean, sigma):
        """Fungsi keanggotaan Gaussian"""
        return np.exp(-0.5 * ((x - mean) / sigma) ** 2)

In [18]:
# =====================================================================
# BAGIAN 3: FUZZY INFERENCE SYSTEM - MAMDANI
# =====================================================================

class MamdaniFIS:
    """
    Implementasi Fuzzy Inference System menggunakan metode Mamdani
    """
    
    def __init__(self):
        self.membership_functions = {}
        self.rules = []
        self.setup_membership_functions()
        self.setup_rules()
    
    def setup_membership_functions(self):
        """Definisi fungsi keanggotaan untuk setiap variabel"""
        
        # Fungsi keanggotaan untuk LIFESTYLE_SCORE (1-4)
        self.membership_functions['lifestyle'] = {
            'poor': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 1, 2.5),
            'moderate': lambda x: FuzzyMembershipFunctions.trimf(x, 1.5, 2.5, 3.5),
            'good': lambda x: FuzzyMembershipFunctions.trimf(x, 2.5, 4, 4)
        }
        
        # Fungsi keanggotaan untuk ACADEMIC_STRESS_SCORE (1-5)
        self.membership_functions['academic'] = {
            'low': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 1, 3),
            'medium': lambda x: FuzzyMembershipFunctions.trimf(x, 2, 3, 4),
            'high': lambda x: FuzzyMembershipFunctions.trimf(x, 3, 5, 5)
        }
        
        # Fungsi keanggotaan untuk DEPRESSION_PREDISPOSITION (0-6)
        self.membership_functions['predisposition'] = {
            'low': lambda x: FuzzyMembershipFunctions.trimf(x, 0, 0, 3),
            'medium': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 3, 5),
            'high': lambda x: FuzzyMembershipFunctions.trimf(x, 3, 6, 6)
        }
        
        # Fungsi keanggotaan untuk OUTPUT - DEPRESSION_RISK (0-1)
        self.membership_functions['depression_risk'] = {
            'low': lambda x: FuzzyMembershipFunctions.trimf(x, 0, 0, 0.5),
            'medium': lambda x: FuzzyMembershipFunctions.trimf(x, 0.2, 0.5, 0.8),
            'high': lambda x: FuzzyMembershipFunctions.trimf(x, 0.5, 1, 1)
        }
    
    def setup_rules(self):
        """Definisi aturan fuzzy berdasarkan logika domain"""
        
        # Format: (lifestyle, academic, predisposition) -> depression_risk
        self.rules = [
            # Aturan untuk risiko rendah
            (('good', 'low', 'low'), 'low'),
            (('good', 'low', 'medium'), 'low'),
            (('good', 'medium', 'low'), 'low'),
            (('moderate', 'low', 'low'), 'low'),
            
            # Aturan untuk risiko sedang
            (('good', 'high', 'low'), 'medium'),
            (('good', 'medium', 'medium'), 'medium'),
            (('good', 'low', 'high'), 'medium'),
            (('moderate', 'medium', 'low'), 'medium'),
            (('moderate', 'low', 'medium'), 'medium'),
            (('moderate', 'high', 'low'), 'medium'),
            (('poor', 'low', 'low'), 'medium'),
            
            # Aturan untuk risiko tinggi
            (('good', 'high', 'medium'), 'high'),
            (('good', 'high', 'high'), 'high'),
            (('good', 'medium', 'high'), 'high'),
            (('moderate', 'high', 'medium'), 'high'),
            (('moderate', 'medium', 'high'), 'high'),
            (('moderate', 'high', 'high'), 'high'),
            (('poor', 'medium', 'low'), 'high'),
            (('poor', 'low', 'medium'), 'high'),
            (('poor', 'high', 'low'), 'high'),
            (('poor', 'medium', 'medium'), 'high'),
            (('poor', 'low', 'high'), 'high'),
            (('poor', 'medium', 'high'), 'high'),
            (('poor', 'high', 'medium'), 'high'),
            (('poor', 'high', 'high'), 'high')
        ]
    
    def fuzzify(self, lifestyle, academic, predisposition):
        """Fuzzifikasi input ke nilai keanggotaan"""
        fuzzified = {}
        
        # Fuzzifikasi lifestyle
        fuzzified['lifestyle'] = {
            'poor': self.membership_functions['lifestyle']['poor'](lifestyle),
            'moderate': self.membership_functions['lifestyle']['moderate'](lifestyle),
            'good': self.membership_functions['lifestyle']['good'](lifestyle)
        }
        
        # Fuzzifikasi academic stress
        fuzzified['academic'] = {
            'low': self.membership_functions['academic']['low'](academic),
            'medium': self.membership_functions['academic']['medium'](academic),
            'high': self.membership_functions['academic']['high'](academic)
        }
        
        # Fuzzifikasi predisposition
        fuzzified['predisposition'] = {
            'low': self.membership_functions['predisposition']['low'](predisposition),
            'medium': self.membership_functions['predisposition']['medium'](predisposition),
            'high': self.membership_functions['predisposition']['high'](predisposition)
        }
        
        return fuzzified
    
    def apply_rules(self, fuzzified_inputs):
        """Aplikasi aturan fuzzy menggunakan metode Mamdani"""
        rule_outputs = {'low': [], 'medium': [], 'high': []}
        
        for rule_antecedent, rule_consequent in self.rules:
            lifestyle_term, academic_term, predisposition_term = rule_antecedent
            
            # Hitung strength dari antecedent (menggunakan AND operator - minimum)
            strength = min(
                fuzzified_inputs['lifestyle'][lifestyle_term],
                fuzzified_inputs['academic'][academic_term],
                fuzzified_inputs['predisposition'][predisposition_term]
            )
            
            # Simpan strength untuk consequent
            if strength > 0:
                rule_outputs[rule_consequent].append(strength)
        
        # Agregasi menggunakan maksimum untuk setiap output class
        aggregated_output = {}
        for output_class in ['low', 'medium', 'high']:
            if rule_outputs[output_class]:
                aggregated_output[output_class] = max(rule_outputs[output_class])
            else:
                aggregated_output[output_class] = 0
        
        return aggregated_output
    
    def defuzzify(self, aggregated_output):
        """Defuzzifikasi menggunakan metode centroid"""
        # Range output 0 sampai 1 dengan resolusi 0.01
        output_range = np.arange(0, 1.01, 0.01)
        
        # Hitung membership degree untuk setiap nilai output
        membership_degrees = np.zeros_like(output_range)
        
        for i, output_val in enumerate(output_range):
            # Untuk setiap class output, hitung membership dan ambil maksimum
            low_membership = min(aggregated_output['low'], 
                               self.membership_functions['depression_risk']['low'](output_val))
            medium_membership = min(aggregated_output['medium'], 
                                  self.membership_functions['depression_risk']['medium'](output_val))
            high_membership = min(aggregated_output['high'], 
                                self.membership_functions['depression_risk']['high'](output_val))
            
            membership_degrees[i] = max(low_membership, medium_membership, high_membership)
        
        # Hitung centroid
        if np.sum(membership_degrees) == 0:
            return 0.5  # Default jika tidak ada membership
        
        centroid = np.sum(output_range * membership_degrees) / np.sum(membership_degrees)
        return centroid
    
    def predict(self, lifestyle, academic, predisposition):
        """Prediksi menggunakan sistem fuzzy Mamdani"""
        # Fuzzifikasi
        fuzzified = self.fuzzify(lifestyle, academic, predisposition)
        
        # Aplikasi aturan
        aggregated = self.apply_rules(fuzzified)
        
        # Defuzzifikasi
        output = self.defuzzify(aggregated)
        
        return output

In [19]:
# =====================================================================
# BAGIAN 4: FUZZY INFERENCE SYSTEM - SUGENO
# =====================================================================

class SugenoFIS:
    """
    Implementasi Fuzzy Inference System menggunakan metode Sugeno
    """
    
    def __init__(self):
        self.membership_functions = {}
        self.rules = []
        self.setup_membership_functions()
        self.setup_rules()
    
    def setup_membership_functions(self):
        """Menggunakan fungsi keanggotaan yang sama dengan Mamdani untuk input"""
        # Fungsi keanggotaan untuk LIFESTYLE_SCORE (1-4)
        self.membership_functions['lifestyle'] = {
            'poor': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 1, 2.5),
            'moderate': lambda x: FuzzyMembershipFunctions.trimf(x, 1.5, 2.5, 3.5),
            'good': lambda x: FuzzyMembershipFunctions.trimf(x, 2.5, 4, 4)
        }
        
        # Fungsi keanggotaan untuk ACADEMIC_STRESS_SCORE (1-5)
        self.membership_functions['academic'] = {
            'low': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 1, 3),
            'medium': lambda x: FuzzyMembershipFunctions.trimf(x, 2, 3, 4),
            'high': lambda x: FuzzyMembershipFunctions.trimf(x, 3, 5, 5)
        }
        
        # Fungsi keanggotaan untuk DEPRESSION_PREDISPOSITION (0-6)
        self.membership_functions['predisposition'] = {
            'low': lambda x: FuzzyMembershipFunctions.trimf(x, 0, 0, 3),
            'medium': lambda x: FuzzyMembershipFunctions.trimf(x, 1, 3, 5),
            'high': lambda x: FuzzyMembershipFunctions.trimf(x, 3, 6, 6)
        }
    
    def setup_rules(self):
        """Definisi aturan fuzzy dengan consequent berupa fungsi linear (Sugeno)"""
        
        # Format: (lifestyle, academic, predisposition) -> linear_function_coefficients
        # f(x,y,z) = a*lifestyle + b*academic + c*predisposition + d
        
        self.rules = [
            # Aturan untuk risiko rendah
            (('good', 'low', 'low'), (-0.1, 0.05, 0.05, 0.2)),
            (('good', 'low', 'medium'), (-0.05, 0.05, 0.1, 0.25)),
            (('good', 'medium', 'low'), (-0.05, 0.1, 0.05, 0.3)),
            (('moderate', 'low', 'low'), (0, 0.05, 0.05, 0.3)),
            
            # Aturan untuk risiko sedang
            (('good', 'high', 'low'), (-0.05, 0.15, 0.05, 0.4)),
            (('good', 'medium', 'medium'), (-0.05, 0.1, 0.1, 0.45)),
            (('good', 'low', 'high'), (-0.05, 0.05, 0.15, 0.5)),
            (('moderate', 'medium', 'low'), (0, 0.1, 0.05, 0.45)),
            (('moderate', 'low', 'medium'), (0, 0.05, 0.1, 0.45)),
            (('moderate', 'high', 'low'), (0, 0.15, 0.05, 0.5)),
            (('poor', 'low', 'low'), (0.1, 0.05, 0.05, 0.5)),
            
            # Aturan untuk risiko tinggi
            (('good', 'high', 'medium'), (-0.05, 0.15, 0.1, 0.65)),
            (('good', 'high', 'high'), (-0.05, 0.15, 0.15, 0.7)),
            (('good', 'medium', 'high'), (-0.05, 0.1, 0.15, 0.7)),
            (('moderate', 'high', 'medium'), (0, 0.15, 0.1, 0.7)),
            (('moderate', 'medium', 'high'), (0, 0.1, 0.15, 0.7)),
            (('moderate', 'high', 'high'), (0, 0.15, 0.15, 0.75)),
            (('poor', 'medium', 'low'), (0.1, 0.1, 0.05, 0.7)),
            (('poor', 'low', 'medium'), (0.1, 0.05, 0.1, 0.7)),
            (('poor', 'high', 'low'), (0.1, 0.15, 0.05, 0.75)),
            (('poor', 'medium', 'medium'), (0.1, 0.1, 0.1, 0.75)),
            (('poor', 'low', 'high'), (0.1, 0.05, 0.15, 0.8)),
            (('poor', 'medium', 'high'), (0.1, 0.1, 0.15, 0.85)),
            (('poor', 'high', 'medium'), (0.1, 0.15, 0.1, 0.85)),
            (('poor', 'high', 'high'), (0.1, 0.15, 0.15, 0.9))
        ]
    
    def fuzzify(self, lifestyle, academic, predisposition):
        """Fuzzifikasi input ke nilai keanggotaan (sama dengan Mamdani)"""
        fuzzified = {}
        
        # Fuzzifikasi lifestyle
        fuzzified['lifestyle'] = {
            'poor': self.membership_functions['lifestyle']['poor'](lifestyle),
            'moderate': self.membership_functions['lifestyle']['moderate'](lifestyle),
            'good': self.membership_functions['lifestyle']['good'](lifestyle)
        }
        
        # Fuzzifikasi academic stress
        fuzzified['academic'] = {
            'low': self.membership_functions['academic']['low'](academic),
            'medium': self.membership_functions['academic']['medium'](academic),
            'high': self.membership_functions['academic']['high'](academic)
        }
        
        # Fuzzifikasi predisposition
        fuzzified['predisposition'] = {
            'low': self.membership_functions['predisposition']['low'](predisposition),
            'medium': self.membership_functions['predisposition']['medium'](predisposition),
            'high': self.membership_functions['predisposition']['high'](predisposition)
        }
        
        return fuzzified
    
    def apply_rules_and_defuzzify(self, fuzzified_inputs, lifestyle, academic, predisposition):
        """Aplikasi aturan dan defuzzifikasi menggunakan weighted average (Sugeno)"""
        
        weighted_sum = 0
        total_weight = 0
        
        for rule_antecedent, rule_consequent in self.rules:
            lifestyle_term, academic_term, predisposition_term = rule_antecedent
            
            # Hitung strength dari antecedent (menggunakan AND operator - minimum)
            strength = min(
                fuzzified_inputs['lifestyle'][lifestyle_term],
                fuzzified_inputs['academic'][academic_term],
                fuzzified_inputs['predisposition'][predisposition_term]
            )
            
            if strength > 0:
                # Hitung output dari fungsi linear
                a, b, c, d = rule_consequent
                rule_output = a * lifestyle + b * academic + c * predisposition + d
                
                # Pastikan output dalam range [0, 1]
                rule_output = max(0, min(1, rule_output))
                
                # Weighted sum
                weighted_sum += strength * rule_output
                total_weight += strength
        
        # Defuzzifikasi menggunakan weighted average
        if total_weight == 0:
            return 0.5  # Default jika tidak ada aturan yang aktif
        
        return weighted_sum / total_weight
    
    def predict(self, lifestyle, academic, predisposition):
        """Prediksi menggunakan sistem fuzzy Sugeno"""
        # Fuzzifikasi
        fuzzified = self.fuzzify(lifestyle, academic, predisposition)
        
        # Aplikasi aturan dan defuzzifikasi
        output = self.apply_rules_and_defuzzify(fuzzified, lifestyle, academic, predisposition)
        
        return output

In [20]:
# =====================================================================
# BAGIAN 5: EVALUASI DAN PERBANDINGAN SISTEM
# =====================================================================

class FuzzySystemEvaluator:
    """
    Kelas untuk evaluasi dan perbandingan sistem fuzzy
    """
    
    def __init__(self, data):
        self.data = data
        self.mamdani_fis = MamdaniFIS()
        self.sugeno_fis = SugenoFIS()
    
    def prepare_data(self):
        """Persiapan data untuk evaluasi"""
        # Ekstrak fitur input dan target
        X = self.data[['Lifestyle_Score', 'Academic_Stress_Score', 'Depression_Predisposition']].values
        y = self.data['Depression_Numeric'].values
        
        # Split data untuk training dan testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
        
        return X_train, X_test, y_train, y_test
    
    def evaluate_system(self, fis_model, X_test, y_test, system_name):
        """Evaluasi satu sistem fuzzy"""
        print(f"\n=== EVALUASI SISTEM {system_name.upper()} ===")
        
        # Prediksi menggunakan sistem fuzzy
        y_pred_fuzzy = []
        
        for i in range(len(X_test)):
            lifestyle, academic, predisposition = X_test[i]
            fuzzy_output = fis_model.predict(lifestyle, academic, predisposition)
            
            # Konversi output fuzzy ke klasifikasi binary
            # Threshold 0.5: jika > 0.5 maka depresi (1), jika <= 0.5 maka tidak depresi (0)
            binary_prediction = 1 if fuzzy_output > 0.5 else 0
            y_pred_fuzzy.append(binary_prediction)
        
        # Hitung akurasi
        accuracy = accuracy_score(y_test, y_pred_fuzzy)
        
        # Tampilkan hasil evaluasi
        print(f"Akurasi {system_name}: {accuracy:.4f} ({accuracy*100:.2f}%)")
        
        # Classification report
        print(f"\nClassification Report {system_name}:")
        print(classification_report(y_test, y_pred_fuzzy, target_names=['No Depression', 'Depression']))
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred_fuzzy)
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['No Depression', 'Depression'],
                    yticklabels=['No Depression', 'Depression'])
        plt.title(f'Confusion Matrix - {system_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
        return accuracy, y_pred_fuzzy
    
    def compare_systems(self):
        """Perbandingan kedua sistem fuzzy"""
        print("\n" + "="*60)
        print("PERBANDINGAN SISTEM FUZZY MAMDANI VS SUGENO")
        print("="*60)
        
        # Persiapan data
        X_train, X_test, y_train, y_test = self.prepare_data()
        
        # Evaluasi sistem Mamdani
        acc_mamdani, pred_mamdani = self.evaluate_system(self.mamdani_fis, X_test, y_test, "Mamdani")
        
        # Evaluasi sistem Sugeno
        acc_sugeno, pred_sugeno = self.evaluate_system(self.sugeno_fis, X_test, y_test, "Sugeno")
        
        # Perbandingan hasil
        print(f"\n=== PERBANDINGAN HASIL ===")
        print(f"Akurasi Mamdani: {acc_mamdani:.4f} ({acc_mamdani*100:.2f}%)")
        print(f"Akurasi Sugeno:  {acc_sugeno:.4f} ({acc_sugeno*100:.2f}%)")
        
        if acc_mamdani > acc_sugeno:
            print(f"→ Mamdani lebih baik dengan selisih {(acc_mamdani-acc_sugeno)*100:.2f}%")
        elif acc_sugeno > acc_mamdani:
            print(f"→ Sugeno lebih baik dengan selisih {(acc_sugeno-acc_mamdani)*100:.2f}%")
        else:
            print("→ Kedua sistem memiliki akurasi yang sama")
        
        # Visualisasi perbandingan
        self._plot_comparison(acc_mamdani, acc_sugeno, pred_mamdani, pred_sugeno, y_test)
        
        return {
            'mamdani_accuracy': acc_mamdani,
            'sugeno_accuracy': acc_sugeno,
            'mamdani_predictions': pred_mamdani,
            'sugeno_predictions': pred_sugeno,
            'actual': y_test
        }
    
    def _plot_comparison(self, acc_mamdani, acc_sugeno, pred_mamdani, pred_sugeno, y_test):
        """Visualisasi perbandingan kedua sistem"""
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Plot 1: Perbandingan Akurasi
        systems = ['Mamdani', 'Sugeno']
        accuracies = [acc_mamdani, acc_sugeno]
        colors = ['skyblue', 'lightcoral']
        
        bars = axes[0].bar(systems, accuracies, color=colors, alpha=0.7, edgecolor='black')
        axes[0].set_title('Perbandingan Akurasi Sistem Fuzzy', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Akurasi')
        axes[0].set_ylim(0, 1)
        
        # Tambahkan nilai akurasi di atas bar
        for bar, acc in zip(bars, accuracies):
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                        f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # Plot 2: Distribusi Prediksi Mamdani
        pred_counts_mamdani = [list(pred_mamdani).count(0), list(pred_mamdani).count(1)]
        axes[1].pie(pred_counts_mamdani, labels=['No Depression', 'Depression'], 
                   autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'lightcoral'])
        axes[1].set_title('Distribusi Prediksi - Mamdani', fontsize=14, fontweight='bold')
        
        # Plot 3: Distribusi Prediksi Sugeno
        pred_counts_sugeno = [list(pred_sugeno).count(0), list(pred_sugeno).count(1)]
        axes[2].pie(pred_counts_sugeno, labels=['No Depression', 'Depression'], 
                   autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'lightcoral'])
        axes[2].set_title('Distribusi Prediksi - Sugeno', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    def analyze_feature_importance(self):
        """Analisis pentingnya fitur dalam prediksi"""
        print(f"\n=== ANALISIS PENTINGNYA FITUR ===")
        
        # Hitung korelasi antara fitur dengan target
        correlation_data = self.data[['Lifestyle_Score', 'Academic_Stress_Score', 
                                     'Depression_Predisposition', 'Depression_Numeric']]
        
        plt.figure(figsize=(10, 8))
        correlation_matrix = correlation_data.corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='RdYlBu_r', center=0,
                   square=True, fmt='.3f')
        plt.title('Korelasi Antar Fitur dan Target', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Tampilkan korelasi dengan target
        target_corr = correlation_matrix['Depression_Numeric'].abs().sort_values(ascending=False)
        print("Korelasi fitur dengan Depression (diurutkan):")
        for feature, corr in target_corr.items():
            if feature != 'Depression_Numeric':
                print(f"  {feature}: {corr:.3f}")

In [21]:
# =====================================================================
# BAGIAN 6: MAIN PROGRAM - TEMPLATE UNTUK EKSEKUSI
# =====================================================================

def main():
    """
    Fungsi utama untuk menjalankan seluruh sistem
    """
    
    print("="*70)
    print("FUZZY INFERENCE SYSTEM UNTUK PREDIKSI DEPRESI MAHASISWA")
    print("Perbandingan Metode Mamdani vs Sugeno")
    print("="*70)
    
    # STEP 1: Load dan preprocess data
    print("\nSTEP 1: LOADING DAN PREPROCESSING DATA")
    print("-" * 40)
    
    # Inisialisasi data preprocessor
    data_processor = DataPreprocessor("uas/Depression Student Dataset.csv")
    
    # Load data
    raw_data = data_processor.load_data()
    
    # Eksplorasi data
    data_processor.explore_data()
    
    # Preprocessing
    processed_data = data_processor.preprocess_data()
    
    # STEP 2: Setup dan testing sistem fuzzy
    print("\nSTEP 2: SETUP SISTEM FUZZY")
    print("-" * 40)
    
    # Test sistem dengan satu contoh
    print("\nTesting sistem dengan contoh data:")
    sample_lifestyle = 2.5  # Moderate lifestyle
    sample_academic = 3.5   # Medium-high academic stress
    sample_predisposition = 2.0  # Low-medium predisposition
    
    # Test Mamdani
    mamdani_system = MamdaniFIS()
    mamdani_result = mamdani_system.predict(sample_lifestyle, sample_academic, sample_predisposition)
    print(f"Hasil Mamdani: {mamdani_result:.3f}")
    
    # Test Sugeno
    sugeno_system = SugenoFIS()
    sugeno_result = sugeno_system.predict(sample_lifestyle, sample_academic, sample_predisposition)
    print(f"Hasil Sugeno: {sugeno_result:.3f}")
    
    # STEP 3: Evaluasi dan perbandingan sistem
    print("\nSTEP 3: EVALUASI DAN PERBANDINGAN")
    print("-" * 40)
    
    # Inisialisasi evaluator
    evaluator = FuzzySystemEvaluator(processed_data)
    
    # Perbandingan sistem
    comparison_results = evaluator.compare_systems()
    
    # Analisis pentingnya fitur
    evaluator.analyze_feature_importance()
    
    # STEP 4: Kesimpulan
    print(f"\n{'='*70}")
    print("KESIMPULAN")
    print("="*70)
    
    mamdani_acc = comparison_results['mamdani_accuracy']
    sugeno_acc = comparison_results['sugeno_accuracy']
    
    print(f"1. Akurasi Sistem Mamdani: {mamdani_acc:.4f} ({mamdani_acc*100:.2f}%)")
    print(f"2. Akurasi Sistem Sugeno:  {sugeno_acc:.4f} ({sugeno_acc*100:.2f}%)")
    
    if mamdani_acc > sugeno_acc:
        best_system = "Mamdani"
        diff = (mamdani_acc - sugeno_acc) * 100
    elif sugeno_acc > mamdani_acc:
        best_system = "Sugeno"
        diff = (sugeno_acc - mamdani_acc) * 100
    else:
        best_system = "Sama"
        diff = 0
    
    if best_system != "Sama":
        print(f"3. Sistem terbaik: {best_system} (selisih {diff:.2f}%)")
    else:
        print("3. Kedua sistem memiliki performa yang sama")
    
    print(f"\n4. Rekomendasi:")
    if best_system == "Mamdani":
        print("   - Gunakan Mamdani untuk interpretabilitas yang lebih baik")
        print("   - Cocok untuk sistem yang membutuhkan penjelasan detail")
    elif best_system == "Sugeno":
        print("   - Gunakan Sugeno untuk efisiensi komputasi yang lebih baik")
        print("   - Cocok untuk sistem real-time atau embedded systems")
    else:
        print("   - Pilih berdasarkan kebutuhan: Mamdani untuk interpretabilitas,")
        print("     Sugeno untuk efisiensi")
    
    return comparison_results

In [22]:
results = main()

FUZZY INFERENCE SYSTEM UNTUK PREDIKSI DEPRESI MAHASISWA
Perbandingan Metode Mamdani vs Sugeno

STEP 1: LOADING DAN PREPROCESSING DATA
----------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'uas/Depression Student Dataset.csv'